# COM6012 Scalable Machine Learning 2019 - Haiping Lu
# Lab 3: Matrix factorisation for collaborative filtering recommender systems

## Objectives

* Task 1: To finish in the lab session. **Essential**
* Task 2: To finish in the lab session. **Essential**
* Task 3: To explore by yourself. **Optional but recommended**

**Suggested reading**: 
* [Collaborative Filtering in Spark](https://spark.apache.org/docs/2.3.2/ml-collaborative-filtering.html)
* [DataBricks movie recommendations tutorial](https://github.com/databricks/spark-training/blob/master/website/movie-recommendation-with-mllib.md![image.png](attachment:image.png)). [**DataBricks**](https://en.wikipedia.org/wiki/Databricks) is a company founded by the creators of Apache Spark, checking out their latest packages at [their GitHub page](https://github.com/databricks), e.g., [integration with Scikit-learn](https://github.com/databricks/spark-sklearn), [Deep Learning Pipelines for Apache Spark including TensorFlow](https://github.com/databricks/spark-deep-learning)
* [Collaborative Filtering on Wiki](http://en.wikipedia.org/wiki/Recommender_system#Collaborative_filtering)
* [Python API on ALS for recommender system](https://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS)
* Chapter 15 (particularly Section 15.3.2) of [PySpark tutorial](https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf) 

[**Learn PySpark APIs via Pictures**](https://github.com/jkthompson/pyspark-pictures) (**from recommended/discover repositories** in GitHub, i.e., found via **recommender systems**!)

https://github.com/haipinglu/ScalableML/

## Finish Lab 1 and Lab 2 sessions

### You will need to make use of lab 1 and lab 2 notebooks for the Quiz in Lab 4.



## 1. Movie recommendation via collaborative filtering



Basic setup unless using shell

In [1]:
#import findspark
#findspark.init()
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Collaborative Filtering RecSys") \
    .getOrCreate()

sc = spark.sparkContext

### Collaborative filtering
[Collaborative filtering](http://en.wikipedia.org/wiki/Recommender_system#Collaborative_filtering) is a classic approach for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix primarily based on the matrix *itself*.  `spark.ml` currently supports **model-based** collaborative filtering, in which users and products are described by a small set of latent factors that can be used to predict missing entries, using the **alternating least squares (ALS)** algorithm. 

API: `class pyspark.ml.recommendation.ALS(rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10, implicitPrefs=False, alpha=1.0, userCol='user', itemCol='item', seed=None, ratingCol='rating', nonnegative=False, checkpointInterval=10, intermediateStorageLevel='MEMORY_AND_DISK', finalStorageLevel='MEMORY_AND_DISK', coldStartStrategy='nan')`

The following parameters are available:
- *rank*: the number of latent factors in the model (defaults to 10).
- *maxIter* is the maximum number of iterations to run (defaults to 10).
- *regParam*: the regularization parameter in ALS (defaults to 1.0).
- *numUserBlocks*/*numItemBlocks*: the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).
- *implicitPrefs*: whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
- *alpha*: a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).
- *nonnegative*: whether or not to use nonnegative constraints for least squares (defaults to false).
- *coldStartStrategy*: can be set to “drop” in order to drop any rows in the DataFrame of predictions that contain NaN values (defaults to "nan", assigning NaN to a user and/or item factor is not present in the model.

### Movie recommendation

In the cells below, we present a small example of collaborative filtering with the data taken from the [MovieLens](http://grouplens.org/datasets/movielens/) project. In this notebook, we use the old 100k dataset (already downloaded in the `Data` folder but you are encouraged to view the source.

The dataset looks like this:

    196     242     3       881250949
    186     302     3       891717742
    22      377     1       878887116
    244     51      2       880606923
    ...

This is a **tab separated** list of 
    
    user id | item id | rating | timestamp 

####  Explicit vs. implicit feedback

The data above is typically viewed as a user-item matrix with the ratings as the entries and users and items determine the row and column indices. The ratings are **explicit feedback**. The *Mean Squared Error* of rating prediction can be used to evaluate the recommendation model.

The ratings can also be used differently. We can treat them as  treated as numbers representing the strength in observations of user actions, i.e., as **implicit feedback** similar to the number of clicks, or the cumulative duration someone spent viewing a movie. Such numbers are then related to the level of confidence in observed user preferences, rather than explicit ratings given to items. The model then tries to find latent factors that can be used to predict the expected preference of a user for an item.

#### Cold-start problem

The cold-start problem refers to the cases when some users and/or items in the test dataset were not present during training the model. In Spark, these users and items are either assigned `NaN` (not a number, default) or dropped (option `drop`).


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

Read the data in and split words (tab separated)

In [3]:
lines = spark.read.text("Data/MovieLens100k.data").rdd
parts = lines.map(lambda row: row.value.split("\t"))

We need to convert the text (`String`) into numbers (`int` or `float`) and then convert RDD to DataFrame

In [4]:
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

[The warning is benign](https://stackoverflow.com/questions/40845304/runtimewarning-numpy-dtype-size-changed-may-indicate-binary-incompatibility)

Check data

In [ ]:
ratings.show(5)

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
|     51|   2.0|880606923|   244|
|    346|   1.0|886397596|   166|
+-------+------+---------+------+
only showing top 5 rows



Check data type

In [ ]:
ratings.printSchema()

root
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- userId: long (nullable = true)



Prepare the training/test data.

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])

Build the recommendation model using ALS on the training data. Note we set cold start strategy to `drop` to ensure we don't get NaN evaluation metrics.

In [ ]:
als = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Evaluate the model by computing the RMSE on the test data

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.9212305273654956


Generate top 10 movie recommendations for each user

In [ ]:
userRecs = model.recommendForAllUsers(10)

In [ ]:
userRecs.show(5,  False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                       |
+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|471   |[[793, 5.059724], [1450, 4.750056], [344, 4.5776315], [422, 4.4951153], [1589, 4.4870133], [1302, 4.4721546], [465, 4.409152], [686, 4.382967], [667, 4.3724327], [1006, 4.3493056]]  |
|463   |[[1166, 4.578314], [1240, 4.4104204], [958, 4.3217916], [854, 4.3081856], [113, 4.27717], [853, 4.209724], [253, 4.166598], [313, 4.161038], [1266, 4.129875], [1142, 4.1059623]]     |
|833   |[[1368, 5.1118503], [1643, 4.641

Generate top 10 user recommendations for each movie

In [ ]:
movieRecs = model.recommendForAllItems(10)
movieRecs.show(5, False)

Generate top 10 movie recommendations for a specified set of users

In [ ]:
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
users.show()
userSubsetRecs.show(3,False)

Generate top 10 user recommendations for a specified set of movies

In [ ]:
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movies.show()
movieSubSetRecs.show(3,False)

In [ ]:
dfItemFactors=model.itemFactors

In [ ]:
dfItemFactors.show()

**`.describe().show()` is very handy to inspect your (big) data for understanding/debugging. Try to use it more often to see.**

In [ ]:
dfItemFactors.describe().show()

In [ ]:
allmovies = ratings.select(als.getItemCol()).distinct()
allmovies.describe().show()

In [ ]:
1682-1665

**Question**: Why is there a difference of 17?

## 2. Exercise - Further analysis of the MovieLens data (completing two or more questions is considered as completion of this exercise).
* Consider more parameter settings to observe the effecttsm e.g., different values of *rank* and/or *regParam*, `nan` vs `drop` for `coldStartStrategy`, etc.
* Use cross validation to select the best model among various parameter settings (reference: Section 15.3.2 of [PySpark tutorial](https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf) )
* Create a standalone program that carries out collaborative filtering. Run this on a bigger [MovieLens dataset](http://grouplens.org/datasets/movielens/), e.g., 1M, 10M or 20M.

* Use 10-fold cross validation (with an 80% training and 20% testing split) to find an average mean average (or squared) error on your test data. Keep your program as parallel as possible. You can create your splits randomly (or any other way you choose!), and don't forget who has access to various variables and who doesn't...

## 3. More Recommender Systems via ALS (Optional but recommended)

### Databricks tutorial
* Complete the tasks in [quiz provided by DataBricks](https://github.com/databricks/spark-training/blob/master/machine-learning/python/MovieLensALS.py) on their data or the data from MovieLens directly. [Solution](https://github.com/databricks/spark-training/blob/master/machine-learning/python/solution/MovieLensALS.py) is posted but you are suggested to try before consulting the solution.

### Santander Kaggle competition on produce recommendation
* A recent Kaggle competition on [Santander Product Recommendation](https://www.kaggle.com/c/santander-product-recommendation) with a prize of **USD 60,000**, and **1,787 teams** participating. 
* Follow this [PySpark notebook on an ALS-based solution](https://www.elenacuoco.com/2016/12/22/alternating-least-squares-als-spark-ml/)
* Learn the way to consider **implcit preferences** and do the same for other recommendation problems.


### Stock Portfolio Recommendations
* Follow Chapter 15 of [PySpark tutorial](https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf)  to perform [Stock Portfolio Recommendations](https://en.wikipedia.org/wiki/Portfolio_investment))
* The data can be downloaded from [Online Retail Data Set](https://archive.ics.uci.edu/ml/datasets/online+retail) at UCI. 
* Please pay attention to the **data cleaning** step that removes rows containing null value. You may need to do the same when you are dealing with real data.
* The data manipulation steps are useful to learn.

### Context-aware recommendation

* See the method in [Joint interaction with context operation for collaborative filtering](https://www.sciencedirect.com/science/article/pii/S0031320318304242?dgcid=rss_sd_all) and implement it in PySpark
* Perform the **time split recommendation** as disscussed in the paper for the above recommender systems.
